In [1]:
import os 
import sys
import time
import datetime
import dateutils
import collections
import itertools
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import scipy as sci
from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
matplotlib.rcParams['figure.figsize'] = (10, 7)
sns.set_style('whitegrid')

* __timestamp__ - дата и время совершения события

* __application_id__- идентификатор приложения

* __client__ - Идентификатор клиента 

* __session_id__ - Идентификатор сессии

* __event_type__ - Тип события

* __event_category__ - Категория события

* __event_name__ - Имя события

* __event_label__ - Дополнительный атрибут события
 
* __device_screen_name__ - Имя экрана на котором произошло событие

* __timezone__ - Часовой пояс

* __device_is_webview__ - Флаг того что страница открыта внутри webview
 
* __page_urlhost__ - Домен страницы

* __page_urlpath_full__ - Путь страницы

* __net_connection_type__ - Тип подключения

* __net_connection_tech__ - Технология подключения

- __client_pin__ - Идентификатор клиента 
- __session_id__ - Идентификатор сессии
- __timestamp__ - Время начала сессии
- __target__ - Целевое действие внутри сессии, multi-class переменная

In [3]:
click_0 = pd.read_parquet('alfabattle2_abattle_clickstream/part-00000.parquet')
click_0.drop(columns=['event_label', 'page_urlpath_full'], inplace=True)
click_0.sort_values(['client', 'session_id', 'timestamp'], inplace=True)
click_0.reset_index(inplace=True)
print(click_0.shape)
click_0.head(3)

(12468595, 14)


,index,timestamp,application_id,client,session_id,event_type,event_category,event_name,device_screen_name,timezone,device_is_webview,page_urlhost,net_connection_type,net_connection_tech
0,241683,2020-09-20 10:22:59.350,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,se,Application Lifecycle,Foreground,None,Europe/Moscow,True,None,mobile,HSPA+
1,6872214,2020-09-20 10:23:00.508,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,sv,None,None,SignInActivity,Europe/Moscow,True,None,mobile,HSPA+
2,6750675,2020-09-20 10:23:09.566,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,sv,None,None,MainListOnWidgetsFragment,Europe/Moscow,True,None,mobile,HSPA+


In [4]:
train_part = pd.read_csv('alfabattle2_abattle_train_target.csv', parse_dates=['timestamp'])
train_part.sort_values(['client_pin', 'session_id', 'timestamp'], inplace=True)
train_part.reset_index(inplace=True)
print(train_part.shape)
train_part.head(3)

(5065350, 5)


,index,session_id,client_pin,timestamp,multi_class_target
0,121700,01e2bfc05dda08dd9ea3e881e45858cc,000033b6509acd1c8eb0d06ebd2e1de9,2020-09-04 16:23:23,main_screen
1,3444610,046e7a872bc29e8fd38fab3cd7bb3636,000033b6509acd1c8eb0d06ebd2e1de9,2020-07-27 19:07:38,main_screen
2,3951305,0512ce7ff813662409ca40acbd1d16df,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-22 23:18:45,main_screen


In [5]:
test_part = pd.read_csv('alfabattle2_prediction_session_timestamp.csv', parse_dates=['timestamp'])
print(test_part.shape)
test_part.head(3)

(79268, 2)


,client_pin,timestamp
0,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53
1,90725b54ce77576883813d87749df6bd,2020-08-01 00:02:57
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,2020-08-01 00:03:14


In [6]:
sample_sub = pd.read_csv('alfabattle2_abattle_sample_prediction.csv')
print(sample_sub.shape)
sample_sub.head(3)

(79268, 2)


,client_pin,prediction
0,f0c674b2bb4dc64be607029271d706ec,credit_info
1,90725b54ce77576883813d87749df6bd,credit_info
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,own_transfer


## EDA + feature engineering with click dataset

In [7]:
# +--------------------+--------------------+
# |       min_timestamp|       min_timestamp|
# +--------------------+--------------------+
# |2020-01-01 03:00:...|2020-10-01 01:42:...|
# +--------------------+--------------------+

In [8]:
click_0.isna().mean() # сходится со всей выборкой

index                  0.000000
timestamp              0.000000
application_id         0.000000
client                 0.000000
session_id             0.000000
event_type             0.000000
event_category         0.457352
event_name             0.464078
device_screen_name     0.224852
timezone               0.000000
device_is_webview      0.018056
page_urlhost           0.964894
net_connection_type    0.035106
net_connection_tech    0.438528
dtype: float64

__Важно: в кликстримах на 1108 уникальных клиентов больше чем в таргете и сабмите__

### timezone

In [9]:
locations = click_0.timezone.str.split(pat="/", n=1, expand=True)

In [10]:
zone = list(locations[0].value_counts().head(8).index)
print(zone)

['Europe', 'Asia', 'Africa', 'Australia', 'Pacific', 'Antarctica', 'America', 'Etc']


In [11]:
city = list(locations[1].value_counts().iloc[:29].index)
print(city)

['Moscow', 'Yekaterinburg', 'Krasnoyarsk', 'Samara', 'Minsk', 'Novosibirsk', 'Omsk', 'Vladivostok', 'Irkutsk', 'Kaliningrad', 'Ho_Chi_Minh', 'Volgograd', 'Novokuznetsk', 'Karachi', 'Dubai', 'Yakutsk', 'Bangkok', 'Kolkata', 'Baku', 'Shanghai', 'Magadan', 'Almaty', 'Saratov', 'Kiev', 'Yerevan', 'Addis_Ababa', 'Tbilisi', 'Jakarta', 'Kamchatka']


In [12]:
locations.loc[~ locations[0].isin(zone), 0] = 'Etc'
locations.loc[~ locations[1].isin(city), 1] = 'Other'

In [13]:
locations = pd.concat([click_0.loc[:, ['client', 'session_id']], locations], axis=1)

In [14]:
locations = locations.drop_duplicates(subset='session_id', keep='last').reset_index(drop=True)
locations.rename(columns={0: 'zone', 1: 'city'}, inplace=True)
print(locations.shape)
locations.head(3)

(1004032, 4)


,client,session_id,zone,city
0,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,Europe,Moscow
1,0014a49ec89e3a43098375b107f8ff2e,230bbd2be4ec56aff09c57566b248e86,Europe,Moscow
2,0014a49ec89e3a43098375b107f8ff2e,2ee80c0a0d2a9a37c3c95c001508b198,Europe,Moscow


In [15]:
locations = locations.loc[locations.client.isin(train_part.client_pin.unique())]

In [16]:
click_0.drop(columns='timezone', inplace=True)

### application_id

In [17]:
# click_0.application_id.value_counts()

In [18]:
click_0.loc[click_0.application_id.str.find('retail') != -1, 'application_id'] = 'retail'
click_0.loc[click_0.application_id.isin(['ncl', 'passport_metrics']), 'application_id'] = 'mobile'
click_0.application_id.value_counts()

mobile    12030967
retail      307704
anketa       72290
site         57634
Name: application_id, dtype: int64

### timestamp

In [20]:
click_0.head()

,index,timestamp,application_id,client,session_id,event_type,event_category,event_name,device_screen_name,device_is_webview,page_urlhost,net_connection_type,net_connection_tech
0,241683,2020-09-20 10:22:59.350,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,se,Application Lifecycle,Foreground,None,True,None,mobile,HSPA+
1,6872214,2020-09-20 10:23:00.508,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,sv,None,None,SignInActivity,True,None,mobile,HSPA+
2,6750675,2020-09-20 10:23:09.566,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,sv,None,None,MainListOnWidgetsFragment,True,None,mobile,HSPA+
3,159552,2020-09-20 10:23:20.355,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,se,Application Lifecycle,Background,None,True,None,mobile,HSPA+
4,6750677,2020-09-20 10:23:21.035,mobile,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,sv,None,None,MainListOnWidgetsFragment,True,None,mobile,HSPA+


In [30]:
%%time
durations = click_0.groupby('session_id').apply(lambda group: group.timestamp.iloc[-1] - group.timestamp.iloc[0]
                                               ).reset_index().rename(columns={0: 'duration'})
durations.loc[:, 'duration'] = durations.duration.astype('timedelta64[ms]')
click_0.drop(columns='timestamp', inplace=True)

CPU times: user 30.5 ms, sys: 8.41 ms, total: 38.9 ms
Wall time: 37.8 ms


### event_type

In [153]:
# se - structured event
# sv - screen view
# pv - page view
# ue - user event
# err - error
# click_0.event_type.value_counts()

In [37]:
(click_0.shape[0] / 100000) * 7 / 60

14.546694166666667

In [38]:
%%time
event_types = click_0.groupby('session_id').apply(lambda group: group.event_type.value_counts(normalize=True)
                                                 ).reset_index().pivot(index="session_id", columns="level_1", 
                                                                       values="event_type").reset_index().fillna(0)
# event_types.drop(columns='err', inplace=True)
# click_0.drop(columns='event_type', inplace=True)

CPU times: user 6.98 s, sys: 119 ms, total: 7.1 s
Wall time: 7.14 s


### event_name

In [259]:
# click_0.event_name.value_counts().head(20)
click_0.drop(columns='event_name', inplace=True)

### event_category

In [253]:
cats = list(click_0.event_category.value_counts().head(18).index)
click_0.loc[~ click_0.event_category.isin(cats), 'event_category'] = np.nan
click_0.event_category.value_counts(normalize=True, dropna=False)

NaN                      0.527169
Application Lifecycle    0.169918
SingleStatement          0.056847
Bottom Navigation        0.043737
Main Screen              0.042806
Widget Dashboard         0.031785
Card To Card Transfer    0.016367
Push                     0.015551
ResultScreen             0.014331
All Payments             0.013428
Bank_Offer               0.011321
Chat                     0.010881
Investments              0.009163
Recharge                 0.007399
Credit Info              0.007076
ClientPhoneTransfer      0.006905
Operations               0.005900
user                     0.005327
ReverseCashBack          0.004092
Name: event_category, dtype: float64

In [260]:
event_categories = click_0.groupby('session_id').apply(lambda group: group.event_category.value_counts(normalize=True)
                                                      ).reset_index().pivot(index="session_id", columns="level_1", 
                                                                       values="event_category").reset_index().fillna(0)
click_0.drop(columns='event_category', inplace=True)

### page_urlhost

In [176]:
click_0.page_urlhost.value_counts()

online.alfabank.ru          201946
click.alfabank.ru           105841
anketa.alfabank.ru           66639
alfabank.ru                  56098
ipoteka.alfabank.ru           5678
travel.alfabank.ru            1472
family.alfabank.ru              21
partner.alfabank.ru             16
investments.alfabank.ru         12
private.auth.alfabank.ru         2
Name: page_urlhost, dtype: int64

In [256]:
click_0.loc[:, 'online_alfa_ru'] = False
click_0.loc[:, 'click_alfa_ru'] = False
click_0.loc[:, 'anketa_alfa_ru'] = False
click_0.loc[:, 'alfa_ru'] = False

click_0.loc[click_0.page_urlhost == 'online.alfabank.ru', 'online_alfa_ru'] = True
click_0.loc[click_0.page_urlhost == 'click.alfabank.ru', 'click_alfa_ru'] = True
click_0.loc[click_0.page_urlhost == 'anketa.alfabank.ru', 'anketa_alfa_ru'] = True
click_0.loc[click_0.page_urlhost == 'alfabank.ru', 'alfa_ru'] = True

click_0.drop(columns='page_urlhost', inplace=True)

### device_screen_name

In [275]:
nams = list(click_0.device_screen_name.value_counts().head(25).index)
print(nams)

['SignInActivity', 'MainListFragment', 'Main list', 'Single Statement', 'MainListOnWidgetsFragment', 'AllPaymentsFragmentNoTemplates', 'StatementFragment', 'Accounts And Cards Widget', 'Final Payment Simple', 'All Payments', 'Chat', 'Card To Card Transfer', 'Widget Dashboard', 'NotificationsFragment', 'OffersFragment', 'Statements screen', 'AccountDetailActivity', 'Client Phone Transfer', 'Credit Info', 'CreditInfoActivity', 'All payments screen', 'FinalPaymentSimpleFragment', 'CardToCardTransferFragment', 'Recharge', 'WebFeatureFragment']


In [276]:
click_0.loc[~ click_0.device_screen_name.isin(nams), 'device_screen_name'] = np.nan
click_0.device_screen_name.value_counts(normalize=True, dropna=False)

NaN                               0.402120
SignInActivity                    0.098318
MainListFragment                  0.096989
Main list                         0.080121
Single Statement                  0.056861
MainListOnWidgetsFragment         0.053544
AllPaymentsFragmentNoTemplates    0.025244
StatementFragment                 0.022345
Accounts And Cards Widget         0.018447
Final Payment Simple              0.013928
All Payments                      0.013241
Chat                              0.010885
Card To Card Transfer             0.010681
Widget Dashboard                  0.010666
NotificationsFragment             0.010582
OffersFragment                    0.010581
Statements screen                 0.007954
AccountDetailActivity             0.007447
Client Phone Transfer             0.007198
Credit Info                       0.007076
CreditInfoActivity                0.007029
All payments screen               0.006479
FinalPaymentSimpleFragment        0.005789
CardToCardT

### net_connection_type

In [288]:
click_0.groupby('session_id')['net_connection_type'].nunique().sort_values()

session_id
000011b33bb3d581fbbbcdadccb1d18e    1
a68a8af76e34e358fd01c64b44e8f1f7    1
a68a9dab8709c697b259f126ed032b0c    1
a68aa1f8c23ddd52f9429e7d284c8221    1
a68ac3de96914a59012e3431a7988a64    1
                                   ..
5ed5d093cd7d6a0d452bf8146d17c6b8    3
f3b73f5650c57a6b361453519adff7e1    3
dcf5b484fb4f67f652bd73f7259312aa    3
a196c34aaa327fb358bd4998fef75e7b    3
de5e17e47d94bdd0472394c60004ca42    3
Name: net_connection_type, Length: 1004032, dtype: int64

In [290]:
click_0.fillna('offline', inplace=True)

net_types = click_0.loc[:, ['session_id', 'net_connection_type']].drop_duplicates(subset='session_id', keep='last')
click_0.drop(columns='net_connection_type', inplace=True)

### device_is_webview

In [266]:
click_0.device_is_webview.fillna(False, inplace=True)

In [40]:
%%time
click_0.iloc[:100000].groupby('session_id').apply(lambda group: group.loc[:, 'device_is_webview':'alfa_ru'].any(axis=0)
                                   ).reset_index()

### net_connection_tech

In [281]:
click_0.drop(columns='net_connection_tech', inplace=True)

In [299]:
click_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12468595 entries, 241683 to 4212658
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   application_id      object
 1   client              object
 2   session_id          object
 3   device_screen_name  object
 4   device_is_webview   bool  
 5   online_alfa_ru      bool  
 6   click_alfa_ru       bool  
 7   anketa_alfa_ru      bool  
 8   alfa_ru             bool  
dtypes: bool(5), object(4)
memory usage: 855.1+ MB


### Feature engineering

In [33]:
train_part.multi_class_target.unique()

array(['main_screen', 'invest', 'statement', 'phone_money_transfer',
       'own_transfer', 'credit_info', 'chat', 'card2card_transfer',
       'mobile_recharge', 'card_recharge'], dtype=object)

In [40]:
print(train_part.shape)
train_part.head(3)

(5065350, 3)


,client_pin,timestamp,multi_class_target
0,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen
1,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest
2,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement


In [35]:
train_part.drop(columns=['session_id'], inplace=True)

In [48]:
print(test_part.shape)
test_part.head(3)

(79268, 2)


,client_pin,timestamp
0,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53
1,90725b54ce77576883813d87749df6bd,2020-08-01 00:02:57
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,2020-08-01 00:03:14


In [50]:
train_part.sort_values(['client_pin', 'timestamp'], ascending=[True, True], inplace=True)

In [54]:
test_part['multi_class_target'] = 'need_to_predict'
test_part.head()

,client_pin,timestamp,multi_class_target
0,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,need_to_predict
1,90725b54ce77576883813d87749df6bd,2020-08-01 00:02:57,need_to_predict
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,2020-08-01 00:03:14,need_to_predict
3,831bf4c0ecccc55e536b8cfb9153d672,2020-08-01 00:04:44,need_to_predict
4,3f1a5a1c492ce877af833113e59c5797,2020-08-01 00:06:40,need_to_predict


In [55]:
pd.concat([train_part, test_part], axis=0)

,client_pin,timestamp,multi_class_target
68402,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-02 22:31:04,statement
4383589,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-19 18:37:26,main_screen
3951305,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-22 23:18:45,main_screen
3604467,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-27 17:29:57,main_screen
1579222,000033b6509acd1c8eb0d06ebd2e1de9,2020-03-04 22:39:45,main_screen
...,...,...,...
79263,4e46e206d38dcb7af41d9ced20d50ac2,2020-09-30 23:41:47,need_to_predict
79264,5b59bf5bd1e07775b1bc95dd7151d2cd,2020-09-30 23:43:25,need_to_predict
79265,94bcc852dd7f806f6c733056d1b0be7b,2020-09-30 23:50:37,need_to_predict
79266,2061f779928217eea2878fd3b605fafd,2020-09-30 23:53:03,need_to_predict


In [53]:
train_part.iloc[1, 1] < train_part.iloc[2, 1]

True